### Attempt to make, new improved x-ray structure analyser


Could do an initial proximity check to see which residues are nearby each other, then only focus on those for the analysis?


Notes: If among standard 20 amino acids, can split into bb and side chain.

If not, then no split is possible but can be treated as a ligand.

Can also provide option to either provide, 1 or 0, or some scaled value...



Hbond defintion:
- up till 3.5 A, reducing between 2 and 3.5 good.
- 180 +- xxx degrees, towards 180 =better.
- Make a model that swaps between 0 and 1 for this? 


vdW defintion. 
- Can do as before, but do last and only if something between them already not defined. 



In [1]:
from typing import Tuple, Optional, List
import warnings
import itertools
import pandas as pd
import numpy as np


import MDAnalysis
from MDAnalysis import Universe
from MDAnalysis.analysis.hydrogenbonds.hbond_analysis import HydrogenBondAnalysis as HBA
from MDAnalysis.analysis import contacts # TODO perhaps remove and use distances implementation?
from MDAnalysis.analysis import distances

/home/roryc/anaconda3/envs/tools_proj_py3_11/lib/python3.11/site-packages/MDAnalysis/topology/TPRParser.py:161: DeprecationWarning: 'xdrlib' is deprecated and slated for removal in Python 3.13
  import xdrlib


In [2]:
# Amino acid definitions - helps define interaction types.
POSITIVE_SB_RESIDUES = ("LYS", "ARG")
NEGATIVE_SB_RESIDUES = ("GLU", "ASP")
HYDROPHOBIC_RESIDUES = ("ALA", "VAL", "LEU", "ILE", "PRO", "PHE", "CYS")
# CATION PI
# AROMATIC 
# OTHER INTERACTION TYPES. 
STANDARD_20_RESIDUES = ["ALA", "ARG", "ASN", "ASP", "CYS", "GLU", "GLN", "GLY", 
                        "HIS", "ILE", "LEU", "LYS", "MET", "PHE", "PRO", "SER", 
                        "THR", "TRP", "TYR", "VAL"]

In [3]:
parm_file = r"WT_PTP1B_Closed.pdb" # r"PDZ_Apo.pdb"
traj_file = r"WT_PTP1B_Closed.pdb" # r"PDZ_Apo.pdb"
# universe = Universe(parm_file, traj_file)

In [4]:
warnings.filterwarnings("ignore", message="Element information is missing")

In [5]:
universe = Universe(traj_file)
universe

<Universe with 4839 atoms>

### Whole thing together


In [6]:
from tools_proj.contacts.contact_analysis import single_frame_contact_analysis
results = single_frame_contact_analysis(topology_file=parm_file,
                              trajectory_file=parm_file,
                             out_file="doesn't_matter_atm",
                             first_res=1,
                             last_res=298)
results

Setup complete, identifying interactions now.


['GLU1 LYS4 SaltBridge SC-SC',
 'GLU1 GLU5 vdw mc-wc',
 'GLU1 SER241 vdw sc-wc',
 'MET2 PHE6 vdw mc-wc',
 'MET2 MET234 vdw sc-sc',
 'MET2 PRO240 vdw sc-mc',
 'MET2 VAL243 vdw sc-wc',
 'MET2 ILE245 vdw sc-sc',
 'MET2 TYR270 vdw mc-sc',
 'MET2 VAL273 vdw sc-sc',
 'MET2 ILE274 vdw wc-wc',
 'GLU3 GLU7 vdw mc-wc',
 'GLU3 ILE274 vdw wc-sc',
 'LYS4 GLN8 vdw mc-wc',
 'GLU5 ILE9 vdw wc-wc',
 'GLU5 LYS246 SaltBridge SC-SC',
 'GLU5 TYR270 vdw sc-sc',
 'PHE6 ASP10 vdw wc-wc',
 'PHE6 TRP15 PiPi SC-SC',
 'PHE6 ARG267 vdw sc-wc',
 'PHE6 TYR270 PiPi SC-SC',
 'PHE6 LEU271 vdw sc-sc',
 'PHE6 ILE274 vdw sc-sc',
 'GLU7 LYS11 vdw mc-wc',
 'GLN8 SER12 vdw mc-wc',
 'ILE9 GLY13 vdw mc-mc',
 'ILE9 SER14 vdw wc-wc',
 'ILE9 TRP15 vdw wc-wc',
 'ILE9 ILE18 vdw sc-sc',
 'ASP10 TRP15 vdw wc-sc',
 'ASP10 ARG267 SaltBridge SC-SC',
 'SER14 ILE18 vdw wc-wc',
 'TRP15 TYR19 vdw mc-wc',
 'TRP15 ALA263 vdw wc-wc',
 'TRP15 LEU266 vdw sc-wc',
 'TRP15 ARG267 vdw sc-wc',
 'TRP15 TYR270 PiPi SC-SC',
 'ALA16 GLN20 vdw mc-wc',
 'A

In [7]:
len(results)

789

In [8]:
from tools_proj.contacts.contact_analysis import single_frame_contact_analysis
single_frame_contact_analysis(topology_file=parm_file,
                              trajectory_file=parm_file,
                             out_file="doesn't_matter_atm",
                             first_res=20,
                             last_res=250)

Setup complete, identifying interactions now.


['GLN20 HIP24 vdw mc-wc',
 'ASP21 GLU25 vdw wc-wc',
 'ILE22 ALA26 vdw mc-wc',
 'ILE22 LYS246 vdw sc-sc',
 'ILE22 LEU249 vdw sc-sc',
 'ILE22 LEU250 vdw wc-sc',
 'ILE22 ARG253 vdw sc-sc',
 'ILE22 LEU266 vdw sc-sc',
 'ARG23 ARG253 vdw wc-sc',
 'ARG23 GLN261 vdw sc-mc',
 'GLU25 LEU250 vdw wc-sc',
 'ALA26 LEU250 vdw wc-sc',
 'ALA26 ARG253 vdw sc-sc',
 'SER27 PHE51 vdw mc-sc',
 'SER27 ARG253 vdw sc-mc',
 'SER27 LYS254 vdw sc-wc',
 'PHE29 PHE51 PiPi SC-SC',
 'PHE29 HIP53 vdw sc-sc',
 'PHE29 LYS254 vdw sc-wc',
 'PRO30 PHE51 vdw mc-wc',
 'PRO30 ASP52 vdw wc-wc',
 'CYS31 LYS35 vdw mc-wc',
 'CYS31 SER49 vdw sc-wc',
 'CYS31 PRO50 vdw wc-mc',
 'CYS31 PHE51 vdw sc-wc',
 'CYS31 ASP52 vdw mc-wc',
 'CYS31 MET257 vdw sc-sc',
 'ARG32 LEU36 vdw mc-wc',
 'ARG32 ASP52 vdw wc-sc',
 'VAL33 ASN39 vdw wc-sc',
 'VAL33 ASP52 vdw wc-sc',
 'VAL33 ARG55 vdw sc-sc',
 'VAL33 TYR65 vdw wc-sc',
 'ALA34 ASN39 vdw mc-wc',
 'ALA34 ASN43 vdw wc-sc',
 'ALA34 SER49 vdw sc-sc',
 'ALA34 PRO50 vdw sc-wc',
 'ALA34 ASP52 vdw mc-sc

### Salt bridges

Could just do an electrostatics based calculation? Need force field charges though...

In [ ]:
from tools_proj.contacts.salt_bridges import check_for_salt_bridge
check_for_salt_bridge(res_numbers=(64, 55), universe=universe)

In [ ]:
universe.residues

In [ ]:
for res1 in range(1, 298+1):
    for res2 in range(res1, len(universe.residues) + 1): 
        result = check_for_salt_bridge(res_numbers=(res1, res2), universe=universe)
        if result:
            print(result)

Done and works, could also consider a sum of distances to measure total strength.

### Cation-Pi interactions

Using this for definitions: https://www.ncbi.nlm.nih.gov/pmc/articles/PMC8338773/ 


In [ ]:
from tools_proj.contacts.cation_pi import check_for_cation_pi
check_for_cation_pi(res_numbers=(99, 168), universe=universe)

In [ ]:
for res1 in range(1, 298+1):
    for res2 in range(res1, len(universe.residues) + 1): 
        result = check_for_cation_pi(res_numbers=(res1, res2), universe=universe)
        if result:
            print(result)

Happy with this :) 

### Pi-Pi Interactions
Referance for defintions:
https://link.springer.com/article/10.1007/s12539-015-0263-z 
Happy with this now

In [ ]:
from tools_proj.contacts.pi_pi import check_for_pi_pi
check_for_pi_pi(res_numbers=(51, 45), universe=universe)

In [ ]:
for res1 in range(1, 298+1):
    for res2 in range(res1, len(universe.residues) + 1): 
        result = check_for_pi_pi(res_numbers=(res1, res2), universe=universe)
        if result:
            print(result)
# this will print result for res1=res2, but this is handled by the main function it will run inside, so it is okay.

### (Side Chain) Hbonds

Can have a protocol if topology provided, but otherwise, this will work with standard 20 amino acids 
Could the expansion just be that I add the backbone names to each one, and then select from the whole aa and not from the side chain? 
Once a hydrogen bond found, can label it based on the atoms used perhaps?

In [ ]:
SC_HBOND_RES_ATOMS_ACCEPTOR = {"ASN": ["OD1"], 
                                "GLN": ["OE1"], 
                                "SER": ["OG"], 
                                "THR": ["OG1"], 
                                "TYR": ["OH"], 
                                "GLU": ["OE1", "OE2"], 
                                "GLU": ["OE1", "OE2"],
                                "ASP": ["OD1", "OD2"],
                                "ASH": ["OD1", "OD2"],
                                "HIE": ["ND1"],
                                "HID": ["NE2"]
}
# List of possible combinations? 
# Hydrogens that match the pair? 
SC_HBOND_RES_ATOMS_DONOR = {"ASN": [{"ND2": ["HD21", "HD22"]}], 
                            "ARG": [{"NE": ["HE"]}, {"NH1": ["HH11", "HH12"]}, {"NH2": ["HH21", "HH22"]}], 
                            "GLN": [{"NE2": ["HE21", "HE22"]}],
                            "SER": [{"OG": ["HG"]}],
                            "THR": [{"OG1": ["HG1"]}],
                            "TYR": [{"OH": ["HH"]}], 
                            "HIE": [{"NE2": ["HE2"]}], 
                            "HID": [{"ND1": ["HD1"]}], 
                            "HIP": [{"NE2": ["HE2"]}, {"ND1": ["HD1"]}],  
                            "LYS": [{"NZ": ["HZ1", "HZ2", "HZ3"]}], 
                            "LYN": [{"NZ": ["HZ2", "HZ3"]}]
}

# cutoffs. 
HB_DIST_CUTOFF = 3.5 # Donor-acceptor Distance
HB_ANGLE_IDEAL = 180 
HB_ANGLE_TOLERANCE = 45 #

In [ ]:
res1, res2 = 39, 63 # 39, 63 # Hbond, 
res1_mc_sele = "name C CA O N H and resid " + str(res1)
res1_sc_sele = "not name C CA O N H and resid " + str(res1)
res1_mc_atoms = universe.select_atoms(res1_mc_sele)
res1_sc_atoms = universe.select_atoms(res1_sc_sele)

res2_mc_sele = "name C CA O N H and resid " + str(res2)
res2_sc_sele = "not name C CA O N H and resid " + str(res2)
res2_mc_atoms = universe.select_atoms(res2_mc_sele)
res2_sc_atoms = universe.select_atoms(res2_sc_sele)

res1_name, res2_name = res_names[res1-1], res_names[res2-1]
res1_name, res2_name

In [ ]:
# Need to consider that if and elif can both be true, and need to test both! 


if (res1_name in SC_HBOND_RES_ATOMS_ACCEPTOR.keys()) and (res2_name in SC_HBOND_RES_ATOMS_DONOR.keys()):
    hb_potential_acceptor_atoms = SC_HBOND_RES_ATOMS_ACCEPTOR[res1_name]
    hb_potential_donor_h_pairs = SC_HBOND_RES_ATOMS_DONOR[res2_name]

    combinations = list(itertools.product(hb_potential_acceptor_atoms, hb_potential_donor_h_pairs))

    for acceptor, donor in combinations:
        acceptor_atom = res1_sc_atoms.select_atoms("name " + acceptor)
        donor_atom = res2_sc_atoms.select_atoms("name " + list(donor.keys())[0])

        # remove donor_acceptor that fail distance test. 
        d_a_dist = distances.distance_array(acceptor_atom.positions, donor_atom.positions)
        if d_a_dist > HB_DIST_CUTOFF:
            continue

        for hydrogen in list(donor.values())[0]:
            hydrogen_atom = res2_sc_atoms.select_atoms("name " + hydrogen)

            dh_vector = donor_atom.positions[0] - hydrogen_atom.positions[0]
            ha_vector = acceptor_atom.positions[0] - hydrogen_atom.positions[0] 
            dha_angle = angle_between_two_vectors(dh_vector, ha_vector) 

            delta_dha_ideal = min(
                np.abs(dha_angle - HB_ANGLE_IDEAL),
                np.abs(dha_angle - HB_ANGLE_IDEAL - 180)
            )
            if delta_dha_ideal < HB_ANGLE_TOLERANCE:
                print("H-bond found!")
                print(acceptor_atom.names, donor_atom.names, hydrogen_atom.names)

elif (res1_name in SC_HBOND_RES_ATOMS_DONOR.keys()) and (res2_name in SC_HBOND_RES_ATOMS_ACCEPTOR.keys()):
    hb_potential_donor_h_pairs = SC_HBOND_RES_ATOMS_DONOR[res1_name]
    hb_potential_acceptor_atoms = SC_HBOND_RES_ATOMS_ACCEPTOR[res2_name]
    # TODO, logic would be here, for follow up.
    # test_sc_hbond()

else: 
    print("no hbonds possible.")

### Hydrophobic

Requirements:
- 4 Å min distance.
- Between two "hydrophobic" residues 
- The atoms that contact must be non-polar (e.g. carbon).
- The atom should also be from the side-chain.

### Metals?
